In [430]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score, mean_squared_error, adjusted_rand_score
from scipy.cluster.hierarchy import dendrogram
from scipy.stats import kurtosis
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib.patches import Ellipse

# Models
from sklearn.neural_network import MLPClassifier

# Algorithms
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.manifold import MDS

# Tools
from sklearn.model_selection import train_test_split, learning_curve

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [191]:
# Set random Seed
plt.rcParams['xtick.labelsize'] = 8
np.random.seed(888)

In [95]:
'''
The two datasets that will be used to train the models in this project. Also used in project 1.
'''
breast_cancer = datasets.load_breast_cancer()
iris = datasets.load_iris()
breast_cancer_X, breast_cancer_Y = breast_cancer.data, breast_cancer.target
iris_X, iris_Y = iris.data, iris.target

In [434]:
'''
Part 1: Clustering Algorithms
'''
def gaussian_clustering(X, y, data, title, savefig=True):
    n_components = len(np.unique(y))
    
    gmm = GaussianMixture(n_components=n_components)
    gmm.means_init = np.array(
        [X[y== i].mean(axis=0) for i in range(n_components)]
    )
    gmm.fit(X)
    
    labels = gmm.predict(X)
    means = gmm.means_
    covariances = gmm.covariances_
    
    if not savefig:
        return gmm
    else:
        # Define colors for the true labels
        colors = ["navy", "tomato", "darkseagreen"]
        plt.figure(figsize=(8, 6))
        # Plot the data points with different colors for each cluster
        for i in range(n_components):
            cluster_data = X[labels == i]
            plt.scatter(cluster_data[:, 0], cluster_data[:, 1], c=colors[i], label=f'Cluster {i + 1}')

        # Plot ellipses to represent cluster covariances
        for i in range(n_components):
            covariance_matrix = covariances[i][:2, :2]
            eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
            angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
            width, height = 2 * np.sqrt(5.991 * eigenvalues)  # 5.991 corresponds to 95% confidence interval

            ellipse = Ellipse(xy=means[i], width=width, height=height, angle=angle, color=colors[i], alpha=0.2)
            plt.gca().add_patch(ellipse)

        # Plot cluster centers
        plt.scatter(means[:, 0], means[:, 1], c='k', marker='x', s=100)

        plt.xlabel(data.feature_names[0])
        plt.ylabel(data.feature_names[1])
        plt.title(f'Gaussian Mixture Model Clusters for {title}')
        if savefig:
            plt.savefig(fname=f'Gaussian_Mixture_Model_{title}')
        plt.clf()
        
        # Get accuracy score
        accuracy = accuracy_score(y, labels)
        print(f"{title}, GaussianMixture accuracy score: {accuracy}")
        return gmm

def agglomerative_clustering(X, y, data, title, savefig=True):
    # Source: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
    def plot_dendrogram(model, **kwargs):
        # Create linkage matrix and then plot the dendrogram

        # create the counts of samples under each node
        counts = np.zeros(model.children_.shape[0])
        n_samples = len(model.labels_)
        for i, merge in enumerate(model.children_):
            current_count = 0
            for child_idx in merge:
                if child_idx < n_samples:
                    current_count += 1  # leaf node
                else:
                    current_count += counts[child_idx - n_samples]
            counts[i] = current_count

        linkage_matrix = np.column_stack(
            [model.children_, model.distances_, counts]
        ).astype(float)

        # Plot the corresponding dendrogram
        dendrogram(linkage_matrix, **kwargs)
    
    n_clusters = len(np.unique(y))
    model = AgglomerativeClustering(n_clusters=n_clusters, compute_distances=True, linkage='average')
    model.fit(X)
    if not savefig:
        return model
    else:
            
        # Define colors for the true labels
        colors = ["navy", "tomato", "darkseagreen"]

        plt.figure(figsize=(8, 6))
        # Plot the dendrogram
        plot_dendrogram(model, truncate_mode='level', p=5)
        plt.title(f'Hierarchial Clustering Diagram for {title}')
        plt.savefig(fname=f'Agglomerative_Clustering_{title}')
        plt.clf()

        # Get accuracy score
        # Evaluate the clustering using ARI
        ari = adjusted_rand_score(y, model.labels_)
        print(f'Adjusted Rand Index (ARI): {ari}')
        return model

In [365]:
'''
Part 2: Dimensionality Reduction Algorithms
'''

def calculate_VIF(X, transformed_X, data, algorithm):
    df = pd.DataFrame(X, columns=data.feature_names)
    
    # VIF of original X
    vif_og = pd.DataFrame()
    vif_og["Feature"] = df.columns
    vif_og["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

    # Step 3: Calculate VIF after PCA
    # Create a DataFrame from the principal components
    df_after = pd.DataFrame(transformed_X)

    # Calculate VIF for each principal component
    vif_after = pd.DataFrame()
    vif_after["Component/Projection"] = df_after.columns
    vif_after["VIF"] = [variance_inflation_factor(df_after.values, i) for i in range(df_after.shape[1])]

    # Display VIF results
    print(f"VIF before {algorithm}:")
    display(vif_og)

    print(f"VIF after {algorithm}:")
    display(vif_after)
                                                                        
def pca_reduction(X, title, data, savefig=True):
    pca = PCA(n_components=0.95)
    mean = np.mean(X, axis=0)
    std_dev = np.std(X, axis=0)
    X_standardized = (X - mean) / std_dev
    transformed_X = pca.fit_transform(X_standardized)
    if not savefig:
        return transformed_X
    else:
        # VIF Analysis
        print(f'{title} VIF Analysis')
        print('==========================')
        calculate_VIF(X, transformed_X, data, "PCA")
        # Plot the explained variance ratio as a scree plot
        explained_variance_ratio = pca.explained_variance_ratio_
        cumulative_variance = np.cumsum(explained_variance_ratio)

        plt.figure(figsize=(8, 6))
        plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, label='Explained Variance Ratio')
        plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='r', label='Cumulative Variance')
        plt.axhline(y=0.95, color='r', linestyle='-')
        plt.grid(True)
        plt.xlabel('Principal Component')
        plt.ylabel('Variance Explained')
        plt.title(f'Scree Plot for PCA for {title}')
        plt.legend()
        plt.savefig(fname=f'PCA_Scree_{title}')
        plt.clf()

        # Get the eigenvalues
        eigenvalues = pca.explained_variance_
        #print(eigenvalues)
        # Plot the eigenvalue distribution
        plt.figure(figsize=(8, 6))
        plt.bar(range(1, len(eigenvalues) + 1), eigenvalues, color='dodgerblue', alpha=0.7)
        plt.xlabel('Eigenvalues')
        plt.ylabel('Magnitude')
        plt.title(f'Eigenvalue Distribution in PCA for {title}')
        plt.grid(True)
        plt.savefig(fname=f'PCA_Eigenvalue_Distribution_{title}')
        plt.clf()

        # Get important features

        # Source: https://stackoverflow.com/a/50845697/22862849
        # number of components
        n_pcs = pca.components_.shape[0]

        # get the index of the most important feature on EACH component
        most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]

        initial_feature_names = data.feature_names
        # get the names
        most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

        dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

        # build the dataframe
        df = pd.DataFrame(dic.items())
        display(df)
        return transformed_X

def ica_reduction(X, title, data, savefig=True):
    ica = FastICA()
    X_ica = ica.fit_transform(X)
    if not savefig:
        return X_ica
    else:
        # VIF Analysis
        print(f'{title} VIF Analysis')
        print('==========================')
        calculate_VIF(X, X_ica, data, "ICA")

        # Calculate feature loadings
        feature_loadings = ica.mixing_  # Mixing matrix

        # Plot the feature loadings
        plt.figure(figsize=(12, 9))
        plt.imshow(np.abs(feature_loadings), cmap='plasma', aspect='auto')
        plt.colorbar()
        plt.xticks(range(X.shape[1]), data.feature_names, rotation=45)
        plt.yticks(range(X_ica.shape[1]), [f'IC {i + 1}' for i in range(X_ica.shape[1])])
        plt.title('Feature Loadings (Mixing Matrix)')
        plt.savefig(fname=f'ICA_Mixing_Matrix_{title}')
        plt.clf()

        # Calculate kurtosis
        kurtosis_values = kurtosis(X_ica, axis=0)
        # Plot the kurtosis values of each independent component
        plt.figure(figsize=(8, 6))
        plt.bar(range(1, len(kurtosis_values) + 1), kurtosis_values, color='dodgerblue', alpha=0.7)
        plt.xlabel('Independent Component')
        plt.ylabel('Kurtosis Value')
        plt.title(f'Kurtosis Values for {title}')
        plt.grid(True)
        plt.savefig(fname=f'ICA_Kurtosis_{title}')
        plt.clf()
        return X_ica

def random_projection(X, title, data, savefig=True):
    projection = GaussianRandomProjection(n_components=2)
    X_rp = projection.fit_transform(X)
    if not savefig:
        return X_rp
    else:
        # VIF Analysis
        print(f'{title} VIF Analysis')
        print('==========================')
        calculate_VIF(X, X_rp, data, "Random Projection")

        # Specify the range of n_components to test
        n_components_range = range(1, X.shape[1] + 1)
        rmses = []
        # Calculate the explained variance for each n_components value
        for n_components in n_components_range:
            projection = GaussianRandomProjection(n_components=n_components)
            X_rp = projection.fit_transform(X)
            X_reconstructed = np.dot(X_rp, projection.components_)
            mse = mean_squared_error(X, X_reconstructed)
            rmses.append(np.sqrt(mse))

        # Plot the scree plot
        plt.figure(figsize=(8, 6))
        plt.bar(n_components_range, rmses)
        plt.xlabel('Number of Components')
        plt.ylabel('Root Mean Squared Error')
        plt.title(f'RMSE between Reconstructed Data and Original Data for {title}')
        plt.grid(True)
        plt.xticks(n_components_range)
        plt.savefig(f"RP_RMSE_Reconstructed_{title}")
        plt.clf()
        return X_rp

def mds_reduction(X, title, data, savefig=True):
    mds = MDS()
    X_mds = mds.fit_transform(X)
    if not savefig:
        return X_mds
    else:
        # VIF Analysis
        print(f'{title} VIF Analysis')
        print('==========================')
        calculate_VIF(X, X_mds, data, "MDS")

        # Create a scatter plot of the MDS results
        plt.figure(figsize=(8, 6))

        colors = ['navy', 'turquoise', 'darkorange']
        lw = 2
        y = data.target
        target_names = data.target_names
        for color, i, target_name in zip(colors, [0, 1, 2], target_names):
            plt.scatter(X_mds[y == i, 0], X_mds[y == i, 1], color=color, alpha=.8, lw=lw,
                        label=target_name)
        plt.legend(loc='best', shadow=False, scatterpoints=1)
        plt.title(f'MDS of {title}')
        plt.savefig(fname=f'MDS_{title}')
        plt.clf()
        return X_mds

In [381]:
'''
Part 3: Run both clustering algorithms on the transformed X's using PCA, ICA, RP, and MDS
'''
def cluster_on_reduced_data():
    X_list = [iris_X, breast_cancer_X]
    data_list = [iris, breast_cancer]
    transformed_X = {
        'PCA': [],
        'ICA': [],
        'RP': [],
        'MDS': [],
    }
    
    # Get the transformed X
    for X, data in zip(X_list, data_list):
        transformed_X['PCA'].append(pca_reduction(X, "", data, savefig=False))
        transformed_X['ICA'].append(ica_reduction(X, "", data, savefig=False))
        transformed_X['RP'].append(random_projection(X, "", data, savefig=False))
        transformed_X['MDS'].append(mds_reduction(X, "", data, savefig=False))
    
    for algo, new_X in transformed_X.items():
        new_iris = new_X[0]
        new_breast_cancer = new_X[1]
        gaussian_clustering(new_iris, iris_Y, iris, f'Gaussian_Clustering_On_{algo}-Iris')
        agglomerative_clustering(new_breast_cancer, breast_cancer_Y, iris, f'Agglomerative_Clustering_On_{algo}-Breast_Cancer')
    return

In [408]:
'''
Part 4: Run MLPClassifier with the transformed data for Iris dataset
'''
def classification_model(model, train_x, train_y, title, fname):
    # Generate the learning curve data points
    train_sizes, train_scores, test_scores = learning_curve(model, train_x, train_y, cv=5, scoring='accuracy')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="b")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="r")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label="Cross-validation score")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.title(title)
    plt.savefig(fname=fname)
    plt.clf()
    print('Training Scores:')
    print(train_scores_mean)
    print('CV Scores:')
    print(test_scores_mean)
    return

def NN_on_reduced_data():
    transformed_X = {
        'PCA': [],
        'ICA': [],
        'RP': [],
        'MDS': [],
    }
    y = iris_Y
    transformed_X['PCA'].append(pca_reduction(iris_X, "", iris, savefig=False))
    transformed_X['ICA'].append(ica_reduction(iris_X, "", iris, savefig=False))
    transformed_X['RP'].append(random_projection(iris_X, "", iris, savefig=False))
    transformed_X['MDS'].append(mds_reduction(iris_X, "", iris, savefig=False))
    
    NN = MLPClassifier(max_iter=1500, alpha=0.0001, hidden_layer_sizes=(50, 50), solver='lbfgs', \
                       learning_rate='constant', early_stopping=True)
    for algo, new_X in transformed_X.items():
        X = new_X[0]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        print(f"Scores for {algo}")
        classification_model(NN, X_train, y_train, f'MLPClassifier {algo}-Data Learning Curve',f'MLPClassifier_With_{algo}-Iris_Data')
        NN.fit(X_train, y_train)
        y_pred = NN.predict(X_test)
        print(f'Accuracy on test set: {accuracy_score(y_test, y_pred)}')
    return


In [505]:
'''
Part 5: NN using Clusters as new features
'''
def NN_w_cluster_data():
    # Gaussian mixture labels
    gmm = gaussian_clustering(iris_X, iris_Y, iris, "", savefig=False)
    labels = np.array(gmm.predict(iris_X)).reshape((150, 1))
    gmm_X = iris_X.copy()
    gmm_X = np.append(gmm_X, labels, axis=1)


    # Agglomerative clustering labels
    agg = agglomerative_clustering(iris_X, iris_Y, iris, '', savefig=False)
    labels = agg.labels_
    mapping = {
        1: 0,
        0: 1,
        2: 2,
    }
    corrected_labels = np.array([mapping[label] for label in labels]).reshape(150, 1)
    agg_X = iris_X.copy()
    agg_X = np.append(agg_X, corrected_labels, axis=1)
    
    # Init model
    NN = MLPClassifier(max_iter=1500, alpha=0.0001, hidden_layer_sizes=(50, 50), solver='lbfgs', \
                       learning_rate='constant', early_stopping=True)
    print('Training w/ GMM cluster data')
    # Train on GMM data
    gmm_X_train, gmm_X_test, gmm_y_train, gmm_y_test = train_test_split(gmm_X, iris_Y, test_size=0.2)
    classification_model(NN, gmm_X_train, gmm_y_train, f'MLPClassifier GMM Cluster Learning Curve', \
                        f'MLPClassifier_With_GMM_Cluster_Iris_Data')
    NN.fit(gmm_X_train, gmm_y_train)
    y_pred = NN.predict(gmm_X_test)
    print(f'Accuracy on test set: {accuracy_score(gmm_y_test, y_pred)}')
    
    print('=================================')
    print('Training w/ AGG cluster data')
    # Train on AgglomerativeClustering data
    agg_X_train, agg_X_test, agg_y_train, agg_y_test = train_test_split(agg_X, iris_Y, test_size=0.2)
    classification_model(NN, agg_X_train, agg_y_train, f'MLPClassifier AGG Cluster Learning Curve', \
                        f'MLPClassifier_With_AGG_Cluster_Iris_Data')
    NN.fit(agg_X_train, agg_y_train)
    y_pred = NN.predict(agg_X_test)
    print(f'Accuracy on test set: {accuracy_score(agg_y_test, y_pred)}')
    return

In [506]:
if __name__ == "__main__":
    #gaussian_clustering(breast_cancer_X, breast_cancer_Y, breast_cancer, "Breast_Cancer_Dataset")
    #gaussian_clustering(iris_X, iris_Y, iris, "Iris_Dataset")
    #agglomerative_clustering(breast_cancer_X, breast_cancer_Y, breast_cancer, "Breast_Cancer_Dataset")
    #agglomerative_clustering(iris_X, iris_Y, iris, "Iris_Dataset")
    #pca_reduction(breast_cancer_X, 'Breast_Cancer_Dataset', breast_cancer)
    #pca_reduction(iris_X, 'Iris_Dataset', iris)
    #ica_reduction(iris_X, 'Iris_Dataset', iris)
    #ica_reduction(breast_cancer_X, 'Breast_Cancer_Dataset', breast_cancer)
    #random_projection(iris_X, 'Iris_Dataset', iris)
    #random_projection(breast_cancer_X, 'Breast_Cancer_Dataset', breast_cancer)
    #mds_reduction(iris_X, 'Iris_Dataset', iris)
    #mds_reduction(breast_cancer_X, 'Breast_Cancer_Dataset', breast_cancer)
    #cluster_on_reduced_data()
    #NN_on_reduced_data()
    NN_w_cluster_data()

Training w/ GMM cluster data
Training Scores:
[1.         1.         0.98461538 0.98378378 0.9875    ]
CV Scores:
[0.89166667 0.93333333 0.90833333 0.96666667 0.96666667]
Accuracy on test set: 0.9666666666666667
Training w/ AGG cluster data
Training Scores:
[1.         1.         1.         0.99189189 0.98541667]
CV Scores:
[0.76666667 0.91666667 0.95       0.95       0.96666667]
Accuracy on test set: 1.0


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>